# Vehicle Routing Problem 

In [3]:
import csv, time, math, random  
from gurobipy import * 
import networkx as nx # For drawing tours and subtours
from haversine import haversine, Unit # For computing the haversine distance

### Reading data

In [4]:
capitals = {}
with open('/Users/pramesh/Downloads/vrpdata.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # skip header
    for row in reader:
        capitals[row[1], row[0]] = (float(row[2]), float(row[3]))

depot_name = ('New Delhi', 'Delhi')
depot_coord = capitals[('New Delhi', 'Delhi')]

dist_matrix = {}
for c1 in capitals:
    for c2 in capitals:
        dist_matrix[c1, c2] = int(haversine(capitals[c1], capitals[c2], unit=Unit.KILOMETERS))

neighbors = {i:[] for i in capitals}
for i, j in dist_matrix:
    if j not in neighbors[i]:
        neighbors[i].append(j)
    if i not in neighbors[j]:
        neighbors[j].append(i)
        


# Demand data
dem = {i: random.randint(1, 10) for i in capitals if i != depot_name}
Q = 50 # Capacity of one vehicle

colors = ['blue', 'red', 'green', 'magenta', 'black', 'gray',  'pink', 'orange', 'cyan']

In [5]:
# Drawing on the map
# Drawing the tour/subtour
import folium
def draw_tour(edges, color):
    average_lat = sum(lat for lat, lon in capitals.values()) / len(capitals)
    average_lon = sum(lon for lat, lon in capitals.values()) / len(capitals)
    
    mymap = folium.Map(location=[average_lat, average_lon], zoom_start=5)
    
    for c in capitals:
        folium.Circle([capitals[c][0], capitals[c][1]],radius=2000, color='red', fill=True, fill_color='red',  fill_opacity=0.7,popup=c).add_to(mymap)


    for i, j in edges:
        coord1 = (capitals[i][0], capitals[i][1])
        coord2 = (capitals[j][0], capitals[j][1])
        folium.PolyLine([coord1, coord2], color=color[i, j], weight=2.5, opacity=1).add_to(mymap)
        
    return mymap

In [6]:
m = Model()
x = {(i, j): m.addVar(vtype = GRB.BINARY) for i in capitals for j in capitals}
u = {i: m.addVar(vtype=GRB.INTEGER) for i in capitals}

for i in capitals:
    if i != depot_name:
        m.addConstr(sum([x[i, j] for j in capitals if i != j]) == 1)
        m.addConstr(sum([x[j, i] for j in capitals if i != j]) == 1)



for i in capitals:
    if i != depot_name:
        m.addConstr(u[i] <= Q)
        m.addConstr(u[i] >= dem[i])

for i, j in dist_matrix:
    if i != depot_name and j != depot_name:
        m.addConstr(u[i] - u[j] + dem[j] <= Q * (1 - x[i, j]))


obj = sum([dist_matrix[i, j] * x[i, j] for i in capitals for j in capitals])
m.setObjective(obj, sense=GRB.MINIMIZE)
m.Params.MIPGap = 0.20

m.optimize()
print(m.status)
edges  = [k for k in x if x[k].x > 0.4]

Set parameter Username
Academic license - for non-commercial use only - expires 2026-04-15
Set parameter MIPGap to value 0.2
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 837 rows, 812 columns and 3645 nonzeros
Model fingerprint: 0x36d6f4b1
Variable types: 0 continuous, 812 integer (784 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [6e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 62660.000000
Presolve removed 81 rows and 29 columns
Presolve time: 0.01s
Presolved: 756 rows, 783 columns, 3564 nonzeros
Variable types: 0 continuous, 783 integer (756 binary)

Root relaxation: objective 8.333440e+03, 138 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

In [8]:
cities = [k[1] for k in x if x[k].x > 0.4 if k[0] == ('New Delhi', 'Delhi')]
routes ={}
ind = 0
for c in cities:
    routes[ind] = [(('New Delhi', 'Delhi'), c)]
    while c != ('New Delhi', 'Delhi'):
        new_node = [k[1] for k in x if x[k].x > 0.4 if k[0] == c][0]
        routes[ind].append((c, new_node))
        c = new_node
    ind += 1
    
edges = []; color = {}; colorsc =colors.copy()
for k in routes:
    edges += routes[k]
    c = colorsc.pop(0)
    for e in routes[k]:
        color[e] = c
draw_tour(edges, color)

## Clarke-Wright heuristic 

In [9]:
routes = {}; ind = 0
cap_routes = {}
for i in capitals:
    if i != depot_name:
        routes[ind] = [(depot_name, i), (i, depot_name)]
        cap_routes[i] = ind
        ind += 1
def merge_routes(ind1, ind2):
    new_route = routes[ind1][:-1] + [(routes[ind1][-1][0], routes[ind2][0][1])] + routes[ind2][1:]
    return new_route

def evaluate_route_demand(ind):
    return sum([dem[k[1]] for k in routes[ind] if k[1] != depot_name])
    

savings = {}
for i in capitals:
    for j in capitals:
        if i != depot_name and j != depot_name and (j, i) not in savings and i != j:
            savings[i, j] = dist_matrix[depot_name, i] + dist_matrix[depot_name, j] - dist_matrix[i, j]
savings = dict(sorted(savings.items(), key=lambda item: item[1], reverse = True))

SE = list(savings.keys())
while SE:
    i, j = SE.pop(0)
    if len(routes[cap_routes[i]]) == 2 and len(routes[cap_routes[j]]) == 2:
        if evaluate_route_demand(cap_routes[i]) + evaluate_route_demand(cap_routes[j]) < Q:            
            new_route = merge_routes(cap_routes[i], cap_routes[j])
            ind +=1
                        
            for c in [k[1] for k in new_route if k[1] != depot_name]:
                if cap_routes[c] in routes:
                    del routes[cap_routes[c]]
                cap_routes[c] = ind
            routes[ind] = new_route

            
    elif (len(routes[cap_routes[i]]) == 2 and len(routes[cap_routes[j]]) > 2) or (len(routes[cap_routes[i]]) > 2 and len(routes[cap_routes[j]]) == 2):
        if len(routes[cap_routes[i]]) == 2:
            if j == routes[cap_routes[j]][0][1]:
                if evaluate_route_demand(cap_routes[j]) + dem[i] < Q:
                    new_route = merge_routes(cap_routes[i], cap_routes[j])
                    ind +=1
                    
                    routes[ind] = new_route
                    for c in [k[1] for k in new_route if k[1] != depot_name]:
                        if cap_routes[c] in routes:
                            del routes[cap_routes[c]]
                        cap_routes[c] = ind
                        
        else:
            if i == routes[cap_routes[i]][0][1]:
                if evaluate_route_demand(cap_routes[i]) + dem[j] < Q:
                    new_route = merge_routes(cap_routes[j], cap_routes[i])
                    ind +=1
                    
                    for c in [k[1] for k in new_route if k[1] != depot_name]:
                        if cap_routes[c] in routes:
                            del routes[cap_routes[c]]
                        cap_routes[c] = ind
                    routes[ind] = new_route
    
        
        
            


In [10]:
edges = []; color = {}; colorsc =colors.copy()
for k in routes:
    edges += routes[k]
    c = colorsc.pop(0)
    for e in routes[k]:
        color[e] = c
print('cost of tour = %d'%sum([dist_matrix[e] for e in edges]))
draw_tour(edges, color)


cost of tour = 16930
